In [15]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.46"
    "NBitcoin -Version 4.1.0.6"
    "FSharp.Data -Version 2.4.6"
    "Newtonsoft.Json -Version 10.0.3"    
  ]    
#load "Paket.Generated.Refs.fsx"

open System
open NBitcoin
open NBitcoin.Crypto   
open NBitcoin.OpenAsset 
open NBitcoin.Stealth 
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System.Collections.Generic 
open System.Linq
open System.Collections

type Wallet =
    static member PrivateKey: Key = 
        (new Key())
    static member PublicKey (k:Key) = 
        if (k=null) then
          (new Key()).PubKey
        else
          k.PubKey 
          
type Wallet with
     static member BitcoinSecretMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.Main)
            else
                k.GetBitcoinSecret(Network.Main)
     static member BitcoinSecretTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.TestNet)
            else
                k.GetBitcoinSecret(Network.TestNet)    
     static member WifMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.Main)
            else
                k.GetWif(Network.Main)
     static member WifTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.TestNet)
            else
                k.GetWif(Network.TestNet)
type Wallet with
     static member BitCoinAddressMain (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.Main) 
     static member BitCoinAddressTest  (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.TestNet) 
     static member PublicKeyHash (k:Key) = 
        Wallet.PublicKey(k).Hash 
     static member BitCoinAddressMain2 (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.Main) 
     static member BitCoinAddressTest2  (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.TestNet) 
        
type Script = 
   static member PublicKeyHash (Pk:string) = 
       new KeyId(Pk) 
   static member Main  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.Main).ScriptPubKey
   static member Test  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.TestNet).ScriptPubKey 
   static member Main  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.Main).ScriptPubKey 
   static member Test  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.TestNet).ScriptPubKey 
   static member BitCoinAddressMain (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.Main)
   static member BitCoinAddressTest  (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.TestNet)

In [16]:
open FSharp.Data
open FSharp.Data.JsonExtensions

type secretJson = JsonProvider<"""{
  "secret": ""
}""">
      
let sj = secretJson.GetSample()
//printfn "%A" sj 
  
let json_uri = @"/home/nbuser/library/secret.json"
let wwif = secretJson.Load(json_uri).JsonValue?secret.AsString()  

let network = Network.TestNet

let sender_bitcoinPrivateKey:Key  = Key.Parse(wwif, Network.TestNet) 
let sender_address = Script.BitCoinAddressTest sender_bitcoinPrivateKey //Wallet.PublicKeyHash bitcoinPrivateKey

printfn "%A" (network)
printfn "sender %A" (sender_address)
printfn "sender secret %A" (Wallet.BitcoinSecretTest sender_bitcoinPrivateKey)
printfn "sender script %A" (Script.Test sender_bitcoinPrivateKey)

let json_uri_dest = @"/home/nbuser/library/secret_dest.json"
let wwif_dest = secretJson.Load(json_uri_dest).JsonValue?secret.AsString()  


let dest_bitcoinPrivateKey:Key   = Key.Parse(wwif_dest, Network.TestNet) 
let dest_address = Script.BitCoinAddressTest dest_bitcoinPrivateKey  
printfn "destination %A" (dest_address)
printfn "dest secret %A" (Wallet.BitcoinSecretTest dest_bitcoinPrivateKey)
printfn "dest script %A" (Script.Test dest_bitcoinPrivateKey)


TestNet
sender mnnaUEKBKJVFjCqVnVPmwefk6CwuVsyRRj
sender secret cRMR4cGGGCh67WmHnZk7Myy9P4nrEuedRovrD3jSqHg7K3HY5wmv
sender script OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG
destination mtYy84R9Ezc6UxWGfHgMj4TmxXFqSy7SPf
dest secret cVdNBTECyg4Af4reyg4qykBMTq8SkqNboJRKotAvEMdVKnhr3CaK
dest script OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG


In [4]:

let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)

let transactionId = uint256.Parse("516c43b617a6fa6dee5f3e9b34d99365b3b89d1381dc9aebcca1e8822c8cb5f4")

// Query the transaction
let transactionResponse:GetTransactionResponse  = client.GetTransaction(transactionId).Result

let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

let receivedCoins = transactionResponse.ReceivedCoins;
let mutable myCoins:Coin = null 
for coin in receivedCoins do
    if (coin.TxOut.ScriptPubKey =  sender_bitcoinPrivateKey.ScriptPubKey) then
        myCoins<- coin:?>Coin

printfn "%A" myCoins.Amount

let ops = client.GetBalance((Wallet.PublicKeyHash sender_bitcoinPrivateKey), true).Result.Operations

let coins =
        query {
               for op in ops do  
                  select op.ReceivedCoins
        }
        
let coins' = coins |> Seq.map (fun x-> x)

//coins' |> List.toArray() |>  Util.Table

0.55000000


In [9]:
open System
open System.IO
open System.Security.Cryptography

let  GetStream (s:string) =
                let stream = new MemoryStream()
                let writer = new StreamWriter(stream)
                writer.Write(s)
                writer.Flush()
                stream.Position <- 0L
                stream
                
//Print the byte array in a readable format.
let ByteString (b:byte[]) = 
        let mutable i = 0
        let mutable s:string = ""
        for i = 0  to b.Length - 1 do
            s <- s + String.Format("{0:X2}", b.[i])
            if (i % 4 = 3) then
                s <- s+ " "
        s    
        
let ByteToHex bytes = 
    bytes 
    |> Array.map (fun (x : byte) -> System.String.Format("{0:X2}", x))
    |> String.concat System.String.Empty

let message = "SF5128165575"

let Sha256 (doc:string) = 
    let sdoc = GetStream doc
    let sh = new SHA256Managed()
    sh.ComputeHash(sdoc)

let hashValue = Sha256 message    

printfn "%A" hashValue

[|75uy; 208uy; 55uy; 47uy; 197uy; 77uy; 18uy; 89uy; 0uy; 242uy; 6uy; 128uy; 6uy;
  169uy; 110uy; 170uy; 113uy; 106uy; 73uy; 25uy; 229uy; 216uy; 66uy; 245uy;
  197uy; 145uy; 194uy; 11uy; 33uy; 117uy; 98uy; 93uy|]


In [10]:

 
let mutable txBuilder = new TransactionBuilder() 
let mutable tx:Transaction = txBuilder
                                .AddCoins(myCoins) 
                                .AddKeys(sender_bitcoinPrivateKey)
                                .Send(dest_address, new Money(1000m, MoneyUnit.Satoshi))
                                .SetChange(sender_address)
                                .SendFees(new Money(0.001m, MoneyUnit.BTC))
                                .Send(TxNullDataTemplate.Instance.GenerateScriptPubKey(hashValue), Money.Zero)
                                .BuildTransaction(true)

printfn "%A" tx

{
  "hash": "ddb1f4880ddfb37223a5fa9df56ee36c0edaaebbec30d204fe85f23c2883fd2b",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 3,
  "lock_time": 0,
  "size": 268,
  "in": [
    {
      "prev_out": {
        "hash": "516c43b617a6fa6dee5f3e9b34d99365b3b89d1381dc9aebcca1e8822c8cb5f4",
        "n": 0
      },
      "scriptSig": "304402203bf9be3c02ff1734d0b1f5b071e82914dd5b4762958827b36d033daeb24a75c80220266f9ea4349acaa1efd24f304a09c626b546f16d624ffe7b6a7446d9815c5a3a01 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1"
    }
  ],
  "out": [
    {
      "value": "0.54899000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.00001000",
      "scriptPubKey": "OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.00000000",
      "scriptPubKey": "OP_RETURN 4bd0372fc54d125900f2068006a96eaa716a4919e5d842f5c591c20b2175625d"
    }
  ]

In [11]:
let verify = txBuilder.Verify(tx)  //check fully signed
printfn "%A" (verify)

true


[visualizza la transazione](https://live.blockcypher.com/btc-testnet/address/mtYy84R9Ezc6UxWGfHgMj4TmxXFqSy7SPf/)

In [13]:
//printfn "%A" (tx.GetHash().ToString())


In [14]:
//printfn "%A" (tx.Sign(sender_bitcoinPrivateKey,myCoins))

In [ ]:
"d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b"